<a href="https://colab.research.google.com/github/16suryanshrastogi/NextWordPrediction/blob/main/Nextwordpridiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 47s 13ms/step - loss: 6.2472 - accuracy: 0.0758
Epoch 2/100
3010/3010 [==============================] - 23s 8ms/step - loss: 5.5149 - accuracy: 0.1244
Epoch 3/100
3010/3010 [==============================] - 22s 7ms/step - loss: 5.1256 - accuracy: 0.1461
Epoch 4/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.7930 - accuracy: 0.1647
Epoch 5/100
3010/3010 [==============================] - 22s 7ms/step - loss: 4.4865 - accuracy: 0.1818
Epoch 6/100
3010/3010 [==============================] - 23s 8ms/step - loss: 4.1995 - accuracy: 0.2017
Epoch 7/100
3010/3010 [==============================] - 22s 7ms/step - loss: 3.9284 - accuracy: 0.2276
Epoch 8/100
3010/3010 [==============================] - 22s 7ms/step - loss: 3.6731 - accuracy: 0.2558
Epoch 9/100
3010/3010 [==============================] - 22s 7ms/step - loss: 3.4314 - accuracy: 0.2906
Epoch 10/100
3010/3010 [==============================] - 22s 7

In [9]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 27ms/step
I will leave if they came upon my
